Do experiments with e-greedy and with softmax
- compare variance of the policy

TODO Bayesian w/ dropout!

https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-7-action-selection-strategies-for-exploration-d3a97b7cceaf

TODO - deep-rl-tensorflow networks (dueling, plus the structure in network.py (Network class)

In [1]:
import numpy as np
import tensorflow as tf

import sys

sys.path.append('..')

from policies import softmax_policy

/Users/adam/anaconda3/envs/energy_py/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/adam/anaconda3/envs/energy_py/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#  setup for testing
num_actions = 3
test_q_values = np.array([0, 5, -5]).reshape(1, 3)

actions = [-1, 0, -1]

from collections import namedtuple, defaultdict

Result = namedtuple('Result',
                   ['temperature', 'probs', 'entropy', 'index', 'action'])

def run_softmax_policy(sess, q_vals, temp):
    soft_, t, p, e, s = sess.run([soft, temperature, probs, entropy, samples],
                          {q_values: q_vals, temperature: temp})

    a = actions[s[0][0]]
    
    return soft_, t, p, e, s, a

def run_single_expt(sess, q_values, temperature):
    results = defaultdict(list)
    for _ in range(1000):
        soft, t, p, e, s, a = run_softmax_policy(sess, q_values, temperature)
        results['softmax'].append(soft)
        results['temperature'].append(t)

        results['soft_0'] = soft[0][0]
        results['soft_1'] = soft[0][1]
        results['soft_2'] = soft[0][2]
        
        results['probs_0'] = p[0][0]
        results['probs_1'] = p[0][1]
        results['probs_2'] = p[0][2]
        
        results['entropy'].append(e[0])
        results['index'].append(s[0][0])
        results['action'].append(a)
        
    return results

In [3]:
q_values = tf.placeholder(shape=(None, num_actions), dtype=tf.float32)

temperature = tf.placeholder(shape=(),
                         dtype=tf.float32,
                         name='softmax_temperature')

soft, probs, entropy, samples = softmax_policy(q_values, temperature)

In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    soft_, t, p, e, s, a = run_softmax_policy(sess, test_q_values, temp=0.5)
    
soft_, t, p, e, s, a

(array([[4.539787e-05, 9.999546e-01, 2.061060e-09]], dtype=float32),
 array(0.5, dtype=float32),
 array([[-1.0000046e+01, -4.5419773e-05, -2.0000046e+01]], dtype=float32),
 array([0.00049944], dtype=float32),
 array([[1]]),
 0)

In [23]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
 
    temps = [50, 25, 10, 5, 1, 0.5, 0.2]
    
    results = []
    
    for temp in temps:
        results.append(run_single_expt(sess, test_q_values, temp))


In [24]:
import pandas as pd

In [25]:
res = pd.DataFrame().from_dict(results[-1])

In [26]:
res

,action,entropy,index,probs_0,probs_1,probs_2,soft_0,soft_1,soft_2,softmax,temperature
0,0,3.471986e-10,1,-25.0,0.0,-50.0,1.388794e-11,1.0,1.928750e-22,"[[1.3887944e-11, 1.0, 1.9287499e-22]]",0.2
1,0,3.471986e-10,1,-25.0,0.0,-50.0,1.388794e-11,1.0,1.928750e-22,"[[1.3887944e-11, 1.0, 1.9287499e-22]]",0.2
2,0,3.471986e-10,1,-25.0,0.0,-50.0,1.388794e-11,1.0,1.928750e-22,"[[1.3887944e-11, 1.0, 1.9287499e-22]]",0.2
3,0,3.471986e-10,1,-25.0,0.0,-50.0,1.388794e-11,1.0,1.928750e-22,"[[1.3887944e-11, 1.0, 1.9287499e-22]]",0.2
4,0,3.471986e-10,1,-25.0,0.0,-50.0,1.388794e-11,1.0,1.928750e-22,"[[1.3887944e-11, 1.0, 1.9287499e-22]]",0.2
5,0,3.471986e-10,1,-25.0,0.0,-50.0,1.388794e-11,1.0,1.928750e-22,"[[1.3887944e-11, 1.0, 1.9287499e-22]]",0.2
6,0,3.471986e-10,1,-25.0,0.0,-50.0,1.388794e-11,1.0,1.928750e-22,"[[1.3887944e-11, 1.0, 1.9287499e-22]]",0.2
7,0,3.471986e-10,1,-25.0,0.0,-50.0,1.388794e-11,1.0,1.928750e-22,"[[1.3887944e-11, 1.0, 1.9287499e-22]]",0.2
8,0,3.471986e-10,1,-25.0,0.0,-50.0,1.388794e-11,1.0,1.928750e-22,"[[1.3887944e-11, 1.0, 1.9287499e-22]]",0.2
9,0,3.471986e-10,1,-25.0,0.0,-50.0,1.388794e-11,1.0,1.928750e-22,"[[1.3887944e-11, 1.0, 1.9287499e-22]]",0.2
